In [84]:
import pandas as pd
import numpy as np

a_customers = pd.read_csv("/kaggle/input/customer-data-integration-for-a-retail-company/company_a_customers.csv")
b_customers = pd.read_csv("/kaggle/input/customer-data-integration-for-a-retail-company/company_b_customers.csv")

# Task1 :Load and concatenate the two datasets.

In [85]:
a_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customer_id   5 non-null      int64 
 1   first_name    4 non-null      object
 2   last_name     5 non-null      object
 3   email         5 non-null      object
 4   phone_number  5 non-null      object
 5   address       5 non-null      object
 6   signup_date   5 non-null      object
dtypes: int64(1), object(6)
memory usage: 408.0+ bytes


In [86]:
# Concatenate the datasets
customers = pd.concat([a_customers, b_customers], ignore_index=True)

# Display the first few rows of the combined DataFrame
customers.head()

,customer_id,first_name,last_name,email,phone_number,address,signup_date
0,1,John,Doe,john.doe@example.com,(555) 123-4567,123 Elm St,2023-01-15
1,2,Jane,Smith,jane.smith@example.com,555-234-5678,456 Oak St,2023/02/16
2,3,NaN,Williams,jack.williams@example.com,555.345.6789,789 Pine St,15-03-2023
3,4,Susan,Johnson,susan.j@example.com,5551234567,101 Maple St,2023.04.18
4,5,John,Doe,john.doe@example.com,(555) 123-4567,123 Elm St,2023-01-15


In [87]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customer_id   10 non-null     int64 
 1   first_name    9 non-null      object
 2   last_name     10 non-null     object
 3   email         10 non-null     object
 4   phone_number  10 non-null     object
 5   address       10 non-null     object
 6   signup_date   10 non-null     object
dtypes: int64(1), object(6)
memory usage: 688.0+ bytes


# Task2 Remove duplicate records.

In [88]:
customers.duplicated(subset=['email']).sum()

3

In [89]:
# Remove duplicate records based on customer_id and email
customers.drop_duplicates(subset=['email'], keep='first', inplace=True)

# Verify removal of duplicates
customers.duplicated(subset=['email']).sum()


0

# Task3 Inconsistent formatting of names and email addresses.

In [90]:
# Standardize first_name and last_name to title case
customers['first_name'] = customers['first_name'].str.title()
customers['last_name'] = customers['last_name'].str.title()

# Standardize email addresses to lowercase
customers['email'] = customers['email'].str.lower()

# Display the changes
customers


,customer_id,first_name,last_name,email,phone_number,address,signup_date
0,1,John,Doe,john.doe@example.com,(555) 123-4567,123 Elm St,2023-01-15
1,2,Jane,Smith,jane.smith@example.com,555-234-5678,456 Oak St,2023/02/16
2,3,NaN,Williams,jack.williams@example.com,555.345.6789,789 Pine St,15-03-2023
3,4,Susan,Johnson,susan.j@example.com,5551234567,101 Maple St,2023.04.18
5,1,Alice,Brown,alice.b@example.com,555 456 7890,202 Birch St,2023-01-20
6,2,Bob,Davis,bob.davis@example.com,555-567-8901,303 Cedar St,16/02/2023
8,4,Charlie,Clark,charlie.clark@example.com,555.678.9012,404 Spruce St,2023.03.17


In [91]:
# Function to extract first name from email if missing
def extract_first_name_from_email(row):
    if pd.isna(row['first_name']):
        return row['email'].split('.')[0].capitalize()
    return row['first_name']

# Apply the function to the DataFrame
customers['first_name'] = customers.apply(extract_first_name_from_email, axis=1)



In [92]:
import re
from datetime import datetime

# Function to standardize phone number formats
def standardize_phone_number(phone):
    phone = re.sub(r'\D', '', phone)  # Remove all non-digit characters
    if len(phone) == 10:
        return f'{phone[:3]}-{phone[3:6]}-{phone[6:]}'
    else:
        return 'Invalid'

# Apply the phone number standardization function
customers['phone_number'] = customers['phone_number'].apply(standardize_phone_number)

# Function to standardize signup date formats to dd/mm/yyyy
def standardize_signup_date(date):
    try:
        # Try different date formats and convert to desired format
        parsed_date = datetime.strptime(date, '%Y-%m-%d')
    except ValueError:
        try:
            parsed_date = datetime.strptime(date, '%Y/%m/%d')
        except ValueError:
            try:
                parsed_date = datetime.strptime(date, '%d-%m-%Y')
            except ValueError:
                try:
                    parsed_date = datetime.strptime(date, '%Y.%m.%d')
                except ValueError:
                    parsed_date = datetime.strptime(date, '%d/%m/%Y')
    return parsed_date.strftime('%d/%m/%Y')

# Apply the signup date standardization function
customers['signup_date'] = customers['signup_date'].apply(standardize_signup_date)

In [93]:
# Function to format email addresses
def format_email(row):
    first_name = row['first_name'].lower() if pd.notna(row['first_name']) else ''
    last_name = row['last_name'].lower() if pd.notna(row['last_name']) else ''
    return f'{first_name}.{last_name}@example.com'

# Apply the email formatting function
customers['email'] = customers.apply(format_email, axis=1)

In [94]:
customers

,customer_id,first_name,last_name,email,phone_number,address,signup_date
0,1,John,Doe,john.doe@example.com,555-123-4567,123 Elm St,15/01/2023
1,2,Jane,Smith,jane.smith@example.com,555-234-5678,456 Oak St,16/02/2023
2,3,Jack,Williams,jack.williams@example.com,555-345-6789,789 Pine St,15/03/2023
3,4,Susan,Johnson,susan.johnson@example.com,555-123-4567,101 Maple St,18/04/2023
5,1,Alice,Brown,alice.brown@example.com,555-456-7890,202 Birch St,20/01/2023
6,2,Bob,Davis,bob.davis@example.com,555-567-8901,303 Cedar St,16/02/2023
8,4,Charlie,Clark,charlie.clark@example.com,555-678-9012,404 Spruce St,17/03/2023
